In [10]:
%matplotlib widget

In [11]:
import numpy as np

from scipy.ndimage import laplace
from tqdm import tqdm

import seaborn as sns
rocket = sns.color_palette("rocket", as_cmap=True)

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff()

BASESHIFT = np.array([-1, 0, 1])

In [12]:
class State:
    def __init__(self, L, N, q):
        self.L = L # lattice size
        self.N = N # number of anyons
        self.q = q # anyons
        self.Φ = np.zeros((L, L), dtype = np.float32) # field
    def draw(self):
        plt.matshow(self.q)
        plt.colorbar()
        plt.show()

In [13]:
def init_state(L, p_anyon):
    vert_anyons = (np.random.rand(L, L) < p_anyon).astype(np.int32)
    vert_anyons += np.roll(vert_anyons, 1, axis=0)

    horiz_anyons = (np.random.rand(L, L) < p_anyon).astype(np.int32)
    horiz_anyons += np.roll(horiz_anyons, 1, axis=1)
    
    q = (vert_anyons + horiz_anyons) % 2
    N = np.sum(q)
    return State(L, N, q)

In [14]:
def update_field(state, η):
    state.Φ += η / 4 * laplace(state.Φ, mode = 'wrap')
    state.Φ += state.q

In [15]:
def update_anyon(state):
    for x, y in np.transpose(np.nonzero(state.q)):
        idx = x * state.L + y
        shifts = np.add.outer(BASESHIFT * state.L, BASESHIFT).flatten()
        neighborhood = state.Φ.take(shifts + idx, mode = 'wrap')
        neighborhood = neighborhood[1::2]
        direction = neighborhood.argmax()
        if np.random.rand() < 0.5:
            state.q[x, y] -= 1
            state.q[x, y] %= 2
            if direction == 0:
                state.q[(x - 1) % state.L, y] += 1
                state.q[(x - 1) % state.L, y] %= 2
            elif direction == 1:
                state.q[x, (y - 1) % state.L] += 1
                state.q[x, (y - 1) % state.L] %= 2
            elif direction == 2:
                state.q[x, (y + 1) % state.L] += 1
                state.q[x, (y + 1) % state.L] %= 2
            elif direction == 3:
                state.q[(x + 1) % state.L, y] += 1
                state.q[(x + 1) % state.L, y] %= 2
            else:
                raise ValueError("Invalid direction")

In [16]:
L = 100
p_anyon = 0.1
mystate = init_state(L, p_anyon)

c = 2
η = 0.5
T = 20

q_history = []

for i in tqdm(range(T)):
    for j in range(c):
        update_field(mystate, 1)
    update_anyon(mystate)
    q_history.append(mystate.q.copy())

q_history = np.array(q_history)

# 22.03 it/s

100%|██████████| 20/20 [00:00<00:00, 52.16it/s]


In [17]:
fig, ax = plt.subplots()
mat = ax.matshow(q_history[0,:,:], cmap = rocket)
fig.colorbar(mat)

def update(i):
    mat.set_data(q_history[i,:,:])
    return mat
anim = animation.FuncAnimation(fig = fig, func = update, frames = T, interval = 500)
anim

In [18]:
fig, ax = plt.subplots()
mat = ax.matshow(q_history[0,:,:], cmap = rocket)
fig.colorbar(mat)

def update(i):
    if i > 0:
        previous = q_history[i-1::-1,:,:]
        weights = 2 ** np.arange(-1, -i-1, -1, dtype = np.float32)
        history = np.tensordot(previous, weights, axes = (0, 0))
        mat.set_data(np.maximum(q_history[i,:,:], history))
    else:
        mat.set_data(q_history[0,:,:])
    return mat
anim = animation.FuncAnimation(fig = fig, func = update, frames = T, interval = 500)
anim